In [1]:
import os
import numpy as np
import pandas as pd
from pylightgbm.models import GBMRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, train_test_split

In [2]:
shift = 190

In [3]:
os.environ['LIGHTGBM_EXEC'] = r'D:\LightGBM\LightGBM\windows\x64\Release\lightgbm.exe'

In [4]:
train_test = pd.read_csv(r'D:/allstate/train_test.csv').drop('Unnamed: 0', axis=1)

In [5]:
directory = r"D:/allstate/"
ids = pd.read_csv(directory + 'test.csv.zip')['id']
train_loss = pd.read_csv(directory + 'train.csv.zip')['loss']

In [7]:
train_loss.shape

(188318,)

In [8]:
ntrain = 188318

In [9]:
train = train_test[:ntrain]
train_x = train.drop(['id','loss'], axis=1)
test_x = train_test[ntrain:].drop(['id','loss'], axis=1)

In [10]:
train_y = np.log(train_loss + shift)

In [74]:
X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, test_size=0.1, random_state=2016)

In [75]:
X_train.shape

(169486, 725)

In [76]:
clf = GBMRegressor(application='regression',
                   num_iterations=1700,
                   learning_rate=0.03,
                   num_leaves=60,
                   tree_learner='serial',
                   num_threads=8,
                   min_data_in_leaf=220,
                   metric='l1',
                   feature_fraction=0.32,
                   bagging_fraction=1.0,
                   early_stopping_round=50,
                   max_depth=-1,
                   boosting_type='gbdt',
                   verbose=False)
clf.fit(X_train, y_train, test_data=[(X_val, y_val)])

In [77]:
clf.best_round

1564

In [79]:
predicted = clf.predict(X_val)
mean_absolute_error(np.exp(y_val), np.exp(predicted))

1127.6123098693258

1136.386420747905 60 листьев 0.03 0.32 фичей 550 минимум в листе

1135.4316392203232 60 листьев 0.03 0.32 фичей 350 минимум в листе

1135.5466563030836 60 листьев 0.03 0.32 фичей 230 минимум в листе

1135.3686307305636 60 листьев 0.03 0.32 фичей 270 минимум в листе

1135.2852721043394 60 листьев 0.03 0.32 фичей 220 минимум в листе

1135.4733167574175 60 листьев 0.03 0.32 фичей 190 минимум в листе

1135.6312692450049 60 листьев 0.03 0.32 фичей 170 минимум в листе

1137.777036075234 60 листьев 0.03 0.32 фичей

1138.0840900140104 
60 листьев
0.03
0.3 фичей

1140.8355124618313
55 листьев
0.03
1100 итераций

1140.2927809707917
50 листьев
0.03
1100 итераций (не сошелся до конца)

1141.1602644427512
80 листьев
0.03
1100 итераций

1141.1933598094815
100 листьев
0.03
1100 итераций

1140.1933598094815
60 листьев
0.03
1000 итераций (не сошелся до конца)

In [80]:
n_folds = 10
cv_sum = 0
early_stopping = 50
mpred = np.zeros(test_x.shape[0])

In [81]:
metafeature = pd.DataFrame()
kf = KFold(n_folds)
for i, (train_index, valid_index) in enumerate(kf.split(train_x)):
    print('\n Fold %d' % (i+1))
    X_train, X_val = train_x.iloc[train_index], train_x.iloc[valid_index]
    y_train, y_val = train_y.iloc[train_index], train_y.iloc[valid_index]
    
    clf = GBMRegressor(application='regression',
                       num_iterations=2000,
                       learning_rate=0.03,
                       num_leaves=60,
                       tree_learner='serial',
                       num_threads=8,
                       min_data_in_leaf=220,
                       metric='l1',
                       feature_fraction=0.32,
                       bagging_fraction=1.0,
                       early_stopping_round=50,
                       max_depth=-1,
                       boosting_type='gbdt',
                       verbose=False)
    
    clf.fit(X_train, y_train, test_data=[(X_val, y_val)])
    
    scores_val = clf.predict(X_val)
    cv_score = mean_absolute_error(np.exp(y_val), np.exp(scores_val))
    print('eval-MAE: %.6f' % cv_score)
    cv_sum += cv_score
    y_pred = np.exp(clf.predict(test_x)) - shift
    #stacking    
    metafeature2 = pd.DataFrame()
    y_stack = np.exp(scores_val) - shift
    metafeature2['id'] = train.iloc[valid_index]['id']
    metafeature2['loss'] = y_stack
    metafeature = pd.concat([metafeature, metafeature2])
    mpred += y_pred   
mpred /= n_folds


 Fold 1
eval-MAE: 1141.121047

 Fold 2
eval-MAE: 1123.011960

 Fold 3
eval-MAE: 1137.245671

 Fold 4
eval-MAE: 1116.497751

 Fold 5
eval-MAE: 1142.470613

 Fold 6
eval-MAE: 1136.846609

 Fold 7
eval-MAE: 1134.811832

 Fold 8
eval-MAE: 1135.829103

 Fold 9
eval-MAE: 1124.668087

 Fold 10
eval-MAE: 1120.952255


In [82]:
print('eval-MAE: %.6f' % (cv_sum/n_folds))

eval-MAE: 1131.345493


In [93]:
print("Writing results")
result = pd.DataFrame(mpred, columns=['loss'])
result["id"] = ids
result = result.set_index("id")
result.to_csv('gbm_first.csv', index=True, index_label='id')

Writing results


In [84]:
metafeature.to_csv('metafeature_10fold_gbm.csv', index=False)

Кластеризация

In [164]:
from sklearn.cluster import KMeans, MiniBatchKMeans, Birch

In [162]:
clust = MiniBatchKMeans(max_iter=300, n_clusters=5, batch_size=5000, max_no_improvement=20)
clust.fit(train_x)

MiniBatchKMeans(batch_size=5000, compute_labels=True, init='k-means++',
        init_size=None, max_iter=300, max_no_improvement=20, n_clusters=5,
        n_init=3, random_state=None, reassignment_ratio=0.01, tol=0.0,
        verbose=0)

In [163]:
clust.predict(test_x[:50])

array([1, 4, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 4, 2, 0, 1, 4, 1,
       1, 4, 1, 4, 1, 1, 1, 1, 1, 0, 3, 0, 2, 2, 0, 1, 0, 1, 0, 4, 1, 1, 1,
       1, 3, 1, 3])

In [118]:
clust.predict(test_x[:50])

array([4, 3, 0, 2, 1, 1, 1, 4, 4, 4, 1, 1, 1, 1, 1, 2, 1, 3, 1, 2, 0, 3, 4,
       1, 3, 1, 3, 1, 1, 4, 4, 4, 2, 1, 2, 0, 0, 2, 1, 2, 1, 2, 3, 4, 4, 1,
       4, 1, 0, 1])